In [ ]:
!pip install torchvision lightning tensorboard

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import lightning as L
import torch.nn.functional as F

class MNISTModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        # Model architecture using nn.Linear
        self.linear = torch.nn.Linear(28*28, 10)
        self.learning_rate = 0.1
    
    def forward(self, x):
        # Forward pass using the defined linear layer
        x = x.flatten(start_dim=1)        
        return self.linear(x)
    
    def training_step(self, batch, batch_idx):
        # Training logic
        X, y = batch
        output = self(X)
        loss = F.cross_entropy(output, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # Validation logic
        X, y = batch
        output = self(X)
        loss = F.cross_entropy(output, y)
        acc = accuracy(output, y)
        self.log('val_loss', loss)
        self.log('val_acc', acc)
        return {'loss': loss, 'accuracy': acc}
    
    def configure_optimizers(self):
        # Configure optimizer
        optimizer = torch.optim.SGD(self.linear.parameters(), lr=self.learning_rate)
        return optimizer

def accuracy(output, label):
    # Accuracy calculation
    return (output.argmax(1) == label).float().mean()

# Data setup

transform = transforms.Compose([
    transforms.ToTensor()
    ])

dataset = MNIST(root='./data', train=True, 
                            download=True, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

val_dataset = MNIST(root='./data', train=False, 
                            download=True, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model, Trainer and Training
model = MNISTModel()
trainer = L.Trainer(max_epochs=1, log_every_n_steps=10, val_check_interval = 500, accelerator="gpu")
trainer.fit(model, train_loader, val_loader)


In [ ]:
trainer.validate(model,val_loader)

## Wie können wir das Ergebniss verbessern?

* Länger Tranieren (mehr epochen)
* Sinvollere Lernrate (ausprobieren!)
* Größeres Netz (mehr Schichten)
* Aktivierungsfunktionen (zwischen den Schichten!)
* Normalisierung (Batch, Layer, Input)
* Dropout
* SGD mit Momentum
* Adam Optimizer
* Data Augmentation
* ChatGPT Fragen :)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs